### 编写爬虫爬去二手房100页的数据:平米 名字 楼层   并保存到数据库和csv文件

In [78]:
%%time
import threading
import time
import requests
import urllib
import re
import lxml
import time
import random
import fake_useragent
import pymysql
import csv

ua = fake_useragent.UserAgent()

class LianjiaSpider(object):
    def __init__(self):
        self.url = 'https://bj.lianjia.com/ershoufang/pg{}/'
        self.db = pymysql.connect('localhost','lofues','123456','spider')
        self.cursor = self.db.cursor()
        self.info = []
        self.test = 1
        
    def get_html(self,url):
        headers = {'User-Agent':ua.random}
        # 尝试三次，否则换一页地址
        if self.test <= 3:
            try:
                html = requests.get(url=url,headers=headers,timeout=3).text
                self.parse_html(html)
                self.test = 1
            except Exception as e:
                print('获取页面失败',e)
                self.test += 1
                self.get_html(url)
                
    # 解析页面
    def parse_html(self,html):
        lxml_html = lxml.etree.HTML(html)
        lis = lxml_html.xpath('//ul[@class="sellListContent"]//li')
        for li in lis:
            house = []

            # 获取房屋名称
            house_name = li.xpath('.//div[@class="title"]/a/text()')[0].strip()
        #         print('house_name:',house_name)

            # 获取房屋地址
            house_address = li.xpath('.//div[@class="address"]/div/a/text()')[0].strip()
        #         print('house_adress:',house_address)

            # 获取房屋的房间数,面积,朝向,是否精装
            house_info = li.xpath('.//div[@class="houseInfo"]/text()')[0].strip()
        #         print('house_info:',house_info)

            house_info = house_info.split('|')
            if len(house_info) == 5:

                room_number = house_info[1].strip()
                house_area = house_info[2].strip()[:-2]
                house_direcation = house_info[3].strip()
                house_perfect = house_info[4].strip()
        #             print('room_number,area,direction,perfect:',room_number,type(house_area),house_direcation,house_perfect)

            # 获取层数
            house_flood = li.xpath('.//div[@class="flood"]/div/text()')[0].strip()
            house_flood = house_flood.strip(' -')
        #         print('house_flood:',house_flood)

            # 获取小区名称
            district_name = li.xpath('.//div[@class="flood"]/div/a/text()')[0].strip()
        #         print('district_name:',district_name)

            # 获取小区总价
            total_price = li.xpath('.//div[@class="priceInfo"]/div[1]/span/text()')[0].strip()
        #         print('total_price:',type(total_price))

            # 获取小区单价
            unit_price = li.xpath('.//div[@class="priceInfo"]/div[2]/span/text()')[0].strip()
            unit_price = unit_price.split('/')[0][2:-1]
        #         print('unit_price:',type(unit_price))

            house = [house_name,house_address,room_number,house_area,house_direcation,house_perfect,
                    house_flood,district_name,total_price,unit_price]
        #         print(house)
            self.info.append(house)
    
    def save_mysql(self,houses):
        ins = 'insert into lianjiahouse(name,address,room,area,direction,perfect,floor,district,total_price,unit_price) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        try:
            self.cursor.executemany(ins,self.info)
        except Exception as e:
            print(e)
            self.db.rollback()
        self.db.commit()
        
    def save_csv(self,houses):
        header = ['个数','名称','地址','房间数','面积','朝向','是否精装','层数','小区名称','总价','单价']
        with open('lianjiahouses.csv','w',encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(header)
            
            for i,house in enumerate(houses):
                writer.writerow([i+1] + house)
        print('保存csv文件成功')
            
        
            
    def run(self):
        for i in range(1,101):
            print('第{}页正在抓取'.format(i))
            url = self.url.format(i)
            self.get_html(url)
            print('第{}页抓取成功'.format(i))
            time.sleep(random.random())

#         print(self.info)
        print('房源总数：',len(self.info))
#         self.save_mysql(self.info)
        self.save_csv(self.info)
        
        # 关闭数据库连接
        self.cursor.close()
        self.db.close()

spider = LianjiaSpider()
spider.run()
            

第1页正在抓取
第1页抓取成功
第2页正在抓取
第2页抓取成功
第3页正在抓取
第3页抓取成功
第4页正在抓取
第4页抓取成功
第5页正在抓取
第5页抓取成功
第6页正在抓取
第6页抓取成功
第7页正在抓取
第7页抓取成功
第8页正在抓取
第8页抓取成功
第9页正在抓取
第9页抓取成功
第10页正在抓取
第10页抓取成功
第11页正在抓取
第11页抓取成功
第12页正在抓取
第12页抓取成功
第13页正在抓取
第13页抓取成功
第14页正在抓取
第14页抓取成功
第15页正在抓取
第15页抓取成功
第16页正在抓取
第16页抓取成功
第17页正在抓取
第17页抓取成功
第18页正在抓取
第18页抓取成功
第19页正在抓取
第19页抓取成功
第20页正在抓取
第20页抓取成功
第21页正在抓取
第21页抓取成功
第22页正在抓取
第22页抓取成功
第23页正在抓取
第23页抓取成功
第24页正在抓取
第24页抓取成功
第25页正在抓取
第25页抓取成功
第26页正在抓取
第26页抓取成功
第27页正在抓取
第27页抓取成功
第28页正在抓取
第28页抓取成功
第29页正在抓取
第29页抓取成功
第30页正在抓取
第30页抓取成功
第31页正在抓取
第31页抓取成功
第32页正在抓取
第32页抓取成功
第33页正在抓取
第33页抓取成功
第34页正在抓取
第34页抓取成功
第35页正在抓取
第35页抓取成功
第36页正在抓取
第36页抓取成功
第37页正在抓取
第37页抓取成功
第38页正在抓取
第38页抓取成功
第39页正在抓取
第39页抓取成功
第40页正在抓取
第40页抓取成功
第41页正在抓取
第41页抓取成功
第42页正在抓取
第42页抓取成功
第43页正在抓取
第43页抓取成功
第44页正在抓取
第44页抓取成功
第45页正在抓取
第45页抓取成功
第46页正在抓取
第46页抓取成功
第47页正在抓取
第47页抓取成功
第48页正在抓取
第48页抓取成功
第49页正在抓取
第49页抓取成功
第50页正在抓取
第50页抓取成功
第51页正在抓取
第51页抓取成功
第52页正在抓取
第52页抓取成功
第53页正在抓取
第53页抓取成功
第54页正在抓取
第54页抓取成功
第55页正在抓取
第55页抓取成功
第56页正在抓取
第56页抓取成功
第57页正在抓取
第

In [72]:
import csv
f = open('s.x','w')
writter = csv.writer(f)
help(writter.writerow)

Help on built-in function writerow:

writerow(...) method of _csv.writer instance
    writerow(iterable)
    
    Construct and write a CSV record from an iterable of fields.  Non-string
    elements will be converted to string.

